<a href="https://colab.research.google.com/github/KonstantinBurkin/parse-records/blob/main/notebooks/8_transformers_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import

In [6]:
import pandas as pd
import json
import os, sys
from tqdm import tqdm
import numpy as np
import polars as pl
import re
from polars import col as c
from pprint import pprint
from sklearn.model_selection import train_test_split

In [7]:
FILES_DIR = "../data"

### data

In [8]:
path = "data_for_models"
os.makedirs(os.path.join(FILES_DIR, path), exist_ok=True)
file_name = "epic_records_with_target.parquet"
file_path = os.path.join(FILES_DIR, path, file_name)
epic_records_with_target = pl.read_parquet(file_path)

In [9]:
epic_records_with_target["target"].value_counts()

target,count
i32,u32
0,517
1,474


In [10]:
epic_records_with_target.sample(2)

patient_id,gender,amnez_life,amnez_disease,condition,ward_table,full_wards,complication_of_main_disease_desc,main_disease_desc,secondary_disease_desc,complication_of_main_disease_code,main_disease_code,secondary_disease_code,admission_datetime,gospitalisation_type,gospitalisation_delivery,background_disease_desc,background_disease_code,following_admission_datetimes,following_main_disease_codes,target,has_competing_disease,gospitalization_result,main_desease_status,condition_status,age,heart_rate,BMI,right atrium,main_disease_I21,main_disease_I20,secondary_and_background_disease_codes,secondary_and_background_disease_count,secondary_disease_I35,secondary_disease_G20,secondary_disease_S72,secondary_disease_E66,…,secondary_disease_S98,secondary_disease_D00,secondary_disease_G56,secondary_disease_R73,secondary_disease_C61,secondary_disease_M06,secondary_disease_G95,secondary_disease_D45,secondary_disease_C50,secondary_disease_E68,secondary_disease_S52,secondary_disease_J67,secondary_disease_S43,secondary_disease_B20,secondary_disease_N20,secondary_disease_D50,secondary_disease_G90,secondary_disease_Z35,secondary_disease_D35,secondary_disease_R55,secondary_disease_R04,secondary_disease_I80,secondary_disease_B37,secondary_disease_I07,secondary_disease_C16,secondary_disease_D01,secondary_disease_M13,secondary_disease_E14,secondary_disease_Z03,secondary_disease_I12,secondary_disease_I11,secondary_disease_D42,secondary_disease_I25,secondary_disease_D13,secondary_disease_N40,secondary_disease_M19,secondary_disease_M42
str,str,str,str,str,str,str,str,str,str,list[str],str,list[str],date,str,str,str,list[str],list[date],list[str],i32,i32,str,str,str,i64,i64,i64,f64,i32,i32,list[str],i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""506E3D75-49E0-2A4D-E055-000000…","""Мужской""","""АНАМНЕЗ ЖИЗНИ:""","""АНАМНЕЗ ЗАБОЛЕВАНИЯ ПОВЫШЕНИЕ…","""[ 'Жалобы: ЖАЛОБЫ НЕ ПРЕДЪЯВЛЯ…","""{'Название': {0: 'Приемное отд…",""" , Сведения о пребывании пацие…","""ОСН 1 по Killip.""","""ИБС. Нестабильная стенокардия.""","""Ожирение 2 степени. (ИМТ 35 кг…","[""I50.1""]","""I20""","[""E66.9""]",2021-06-18,"""Первичный""","""Плановая госпитализация""","""Гипертоническая болезнь III ст…","[""I11.9""]",[2023-10-27],"[""I20""]",0,0,"""Улучшение""","""Острое""","""Удовлетворительное""",57,69,33,3.4,0,1,"[""E66.9"", ""I11.9"", ""I50.1""]",3,0,0,0,1,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
"""114381736""","""Мужской""","""Б""","""АНАМНЕЗ ЗАБОЛЕВАНИЯ 26.09.17Г …","""[ 'Жалобы: ЖАЛОБЫ НА МОМЕНТ ОС…","""{'Название': {0: 'Приемное отд…",""" , Сведения о пребывании пацие…","""ОСН по Killip I""","""ОКС с подъёмом сегмента ST Пер…","""Пароксизмальная форма ФП,вне п…","[""I50.1""]","""I21""","[""I48.0"", ""I25.3"", … ""U07.1""]",2022-01-25,"""Первичный""","""Плановая госпитализация""","""Гипертоническая болезнь III с…","[""I11.9""]",[2022-03-27],"[""I21""]",1,0,"""Улучшение""","""Острое""","""Удовлетворительное""",50,78,null,3.7,1,0,"[""I48.0"", ""I25.3"", … ""I50.1""]",6,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0


### text features

In [11]:
def search_condition(x):
    found = re.sub(
        r"&quot",
        " ",
        x,
    )
    found = re.sub(
        r"\n",
        " ",
        found,
    )
    found = re.sub(
        r"\(А\)",
        " ",
        found,
    )
    found = re.sub(
        "СО СЛОВ ПАЦИЕНТА",
        " ",
        found,
    )
    found = re.sub(
        r" +",
        " ",
        found,
    )
    return found


epic_records_with_target = epic_records_with_target.with_columns(
    c("amnez_life").replace(old=["Б", "АНАМНЕЗ ЖИЗНИ:"], new=""),
).with_columns(
    c("amnez_life").map_elements(
        search_condition,
        return_dtype=str,
    ),
)
epic_records_with_target["amnez_life"].to_numpy()[:10]

array(['',
       'ЭПИДЕМИОЛОГИЧЕСКИЙ АНАМНЕЗ: , С БОЛЬНЫМИ КОРЬЮ НЕ КОНТАКТИРОВАЛ ., , С БОЛЬНЫМИ ГРИППОМ И ДРУГИМИ ОРВИ НЕ КОНТАКТИРОВАЛ ЗА 2022-2023ГГ., В ТЕЧЕНИЕ ПОСЛЕДНЕГО МЕСЯЦА ЗА ТЕРРИТОРИЮ РФ НЕ ВЫЕЗЖАЛ . , С БОЛЬНЫМ ;ОСПОЙ ОБЕЗЬЯН ; НЕ КОНТАКТИРОВАЛ . ВАКЦИНАЦИЯ ОТ ГРИППА НЕТ ВАКЦИНАЦИЯ ОТ COVID-19 2021 РЕВАКЦИНАЦИЯ ОТ COVID-19 НЕТ В АНАМНЕЗЕ: COVID-19 ДА 2022 ТУБЕРКУЛЕЗ: НЕТ ВИЧ ИНФЕКЦИЯ: НЕТ ВИРУСНЫЕ ГЕПАТИТЫ: НЕТ СИФИЛИС: НЕТ ПЕРЕНЕСЕННЫЕ ЗАБОЛЕВАНИЯ: ГБ ТРАВМЫ, ОПЕРАЦИИ: АППЕНДЭКТОМИЯ В ЮННОСТИ АЛЛЕРГИЧЕСКИЕ РЕАКЦИИ (С УКАЗАНИЕМ ТИПА И ВИДА РЕАКЦИИ): НЕТ ТРАНСФУЗИИ (ПЕРЕЛИВАНИЯ) ДОНОРСКОЙ КРОВИ И (ИЛИ) ЕЕ КОМПОНЕНТОВ: НЕТ Д-УЧЕТ: НЕТ',
       '',
       'ЭПИДЕМИОЛОГИЧЕСКИЙ АНАМНЕЗ: , С БОЛЬНЫМИ КОРЬЮ НЕ КОНТАКТИРОВАЛ ., , С БОЛЬНЫМИ ГРИППОМ И ДРУГИМИ ОРВИ НЕ КОНТАКТИРОВАЛ ЗА 2022-2023ГГ. В 20.10.2023Г ПРИЛИТЕЛ ИЗ ТАДЖИКИСТАНА. ВАКЦИНАЦИЯ ОТ ГРИППА НЕТ ВАКЦИНАЦИЯ ОТ COVID-19 ДА В 2021Г РЕВАКЦИНАЦИЯ ОТ COVID-19 ДА 2022Г В АНАМНЕЗЕ: COVID-19 НЕТ ТУБЕРКУЛЕЗ: НЕТ ВИЧ ИНФЕКЦИЯ:

In [12]:
def search_condition(x):
    found = re.sub(
        "Объективный статус: ОБЪЕКТИВНЫЙ СТАТУС:",
        " ",
        " ".join(eval(x)),
    )
    found = re.sub(
        r"Жалобы:",
        " ",
        found,
    )
    found = re.sub(
        r" +",
        " ",
        found,
    )
    return found


epic_records_with_target = epic_records_with_target.with_columns(
    c("condition").map_elements(
        search_condition,
        return_dtype=str,
    )
)
epic_records_with_target["condition"].to_numpy()[:10]

array([' ЖАЛОБЫ НА МОМЕНТ ОСМОТРА НЕ ПРЕДЪЯВЛЯЕТ. ',
       ' НА ДАВЯЩУЮ БОЛЬ ЗА ГРУДИНОЙ С ИРРАДАЦИЕЙ В ЛЕВУЮ РУКУ ВОЗНИКШАЯ ПОСЛЕ ФИЗИЧЕСКОЙ НАГРУЗКИ ( ПОДЬЕМ НА 4 ЭТАЖ), ДЛИТЕЛЬНО, СОПРОВОЖДАЮЩЕЯСЯ ПОТЛИВОСТЬЮ И СЛАБОСТЬЮ. НА МОМЕНТ ОСМОТРА В ПО ОКД РЕЦИДИВ АНГИНОЗНЫХ БОЛЕЙ. Объективный статус: ФИЗИКАЛЬНОЕ ИССЛЕДОВАНИЕ: СТЕПЕНЬ ТЯЖЕСТИ ТЯЖЕЛОЕ, ПО ЗАБОЛЕВАНИЮ СОЗНАНИЕ: ЯСНОЕ УРОВЕНЬ СОЗНАНИЯ ПО ШКАЛЕ ГЛАЗГО 15 Б СОСТОЯНИЕ ПОДКОЖНО-ЖИРОВОЙ КЛЕТЧАТКИ: ТЕЛОСЛОЖЕНИЕ ГИПЕРСТЕНИК ПИТАНИЕ: ИЗБЫТОЧНОЕ ОЦЕНКА СОСТОЯНИЯ КОЖНЫХ ПОКРОВОВ: ФИЗИОЛОГИЧЕСКОЙ ОКРАСКИ ГИПЕРГИДРОЗ ОЦЕНКА СОСТОЯНИЯ ВИДИМЫХ СЛИЗИСТЫХ ОБОЛОЧЕК: БЕЗ ПАТОЛОГИЧЕСКИХ ВЫСЫПАНИЙ ЗЕВ ЧИСТЫЙ, НЕ ГИПЕРЕМИРОВАН ЯЗЫК ВЛАЖНЫЙ ЧИСТЫЙ ОТЕКИ: ОТСУТСТВУЮТ ЛИМФОУЗЛЫ ПРИ ПАЛЬПАЦИИ: НЕ УВЕЛИЧЕНЫ, НЕ ПАЛЬПИРУЮТСЯ ОЦЕНКА КОСТНО-МЫШЕЧНОЙ СИСТЕМЫ: БЕЗ ПАТОЛОГИЧЕСКИХ ИЗМЕНЕНИЙ; НИЖНИЕ КОНЕЧНОСТИ: ФИЗИОЛОГИЧЕСКОЙ ОКРАСКИ НЕ ГИПЕРЕМИРОВАНЫ НА ОЩУПЬ ТЕПЛЫЕ ПУЛЬСАЦИЯ ОТЧЕТЛИВАЯ ПРИ ПАЛЬПАЦИИ БЕЗБОЛЕЗНЕННЫ РЕЗУЛЬТАТЫ ПЕРКУССИИ И АУСКУЛЬТАЦИИ ЛЕГКИХ:

In [13]:
def search_condition(x):
    found = re.sub(
        r"&quot",
        " ",
        x,
    )
    found = re.sub(
        r"\(А\)",
        " ",
        found,
    )
    found = re.sub(
        r"\n",
        " ",
        found,
    )
    found = re.sub(
        r"!",
        " ",
        found,
    )
    found = re.sub(
        r"\.+",
        " ",
        found,
    )
    found = re.sub(
        r" +",
        " ",
        found,
    )
    return found


epic_records_with_target = epic_records_with_target.with_columns(
    c("amnez_disease").map_elements(
        search_condition,
        return_dtype=str,
    )
)
epic_records_with_target["amnez_disease"].to_numpy()[:10]

array(['АНАМНЕЗ ЗАБОЛЕВАНИЯ КОРОНАРНЫЙ АНАМНЕЗ ОТРИЦАЕТ НАСТОЯЩЕЕ УХУДШЕНИЕ СОСТОЯНИЯ 29 09 2020Г ОКОЛО 20:00, КОГДА ВПЕРВЫЕ В СОСТОЯНИИ ФИЗИЧЕСКОГО ПОКОЯ, ОТМЕТИЛ ПОЯВЛЕНИЕ ДАВЯЩЕЙ БОЛИ ЗА ГРУДИНОЙ, БЕЗ ИРРАДИАЦИИ, ДЛИТЕЛЬНОСТЬЮ ДО 30 МИНУТ, СОПРОВОЖДАЮЩЕЙСЯ ЧУВСТВОМ НЕХВАТКИ ВОЗДУХА, ПОТЛИВОСТЬЮ, ОБЩЕЙ СЛАБОСТЬЮ ВЫЗВАЛ БСМП, ДОСТАВЛЕН В ОКД, ОТ ПРЕДЛОЖЕННОЙ ГОСПИТАЛИЗАЦИИ ПАЦИЕНТ КАТЕГОРИЧЕСКИ ОТКАЗАЛСЯ ПИСЬМЕННО В НАСТОЯЩИЙ МОМЕНТ САМОСТОЯТЕЛЬНО ОБРАТИЛСЯ (АНГИНОЗНЫЕ БОЛИ И ЭКВИВАЛЕНТЫ ЗА ПРОШЕДШИЕ СУТКИ ОТРИЦАЕТ) ',
       'СЧИТАТЕТ СЕБЯ БОЛЬНЫМ С 26 09 2023 ГОДА 03:00 КОГДА ОТМЕТИЛ ПОЯВЛЕНИЕ ДАВЯЩИХ БОЛЕЙ ЗА ГРУДИНОЙ С ИРРАДАЦИЕЙ В ЛЕВУЮ РУКУ ВОЗНИКШАЯ ПОСЛЕ ФИЗИЧЕСКОЙ НАГРУЗКИ ( ПОДЬЕМ НА 4 ЭТАЖ), ДЛИТЕЛЬНО, СОПРОВОЖДАЮЩЕЯСЯ ПОТЛИВОСТЬЮ И СЛАБОСТЬЮ САМОСТОЯТЕЛЬНО ПРИНЯЛ 1 ТАБ КАПТОПРИЛА, БЕЗ КЛИНИЧЕСКОГО ЭФФЕКТА ОБРАТИЛСЯ В МЕДПУНКТ АЭРОПОРТА ПО ЭКГ: ИШЕМИЧЕСКИЕ ИЗМЕНЕНИЯ ОКАЗАНА ИПОМОЩЬ ДАНО НАПРАВЛЕНИЕ НА ГОСПИТАЛИЗАЦИЮ В ОКД С ДИАГНОЗОМ: ИБС ВПЕРВЫЕ ВОЗНИКШАЯ СТЕНОКАРДИЯ ВЫЗВА

In [14]:
epic_records_with_target["main_disease_desc"].to_numpy()[0]

'ИБС. Острый инфаркт миокарда передне-перегородочной, верхушечно-боковой стенок ЛЖ без подъема сегмента ST от 29.07.2020 г.'

In [15]:
epic_records_with_target.sample(3)

patient_id,gender,amnez_life,amnez_disease,condition,ward_table,full_wards,complication_of_main_disease_desc,main_disease_desc,secondary_disease_desc,complication_of_main_disease_code,main_disease_code,secondary_disease_code,admission_datetime,gospitalisation_type,gospitalisation_delivery,background_disease_desc,background_disease_code,following_admission_datetimes,following_main_disease_codes,target,has_competing_disease,gospitalization_result,main_desease_status,condition_status,age,heart_rate,BMI,right atrium,main_disease_I21,main_disease_I20,secondary_and_background_disease_codes,secondary_and_background_disease_count,secondary_disease_I35,secondary_disease_G20,secondary_disease_S72,secondary_disease_E66,…,secondary_disease_S98,secondary_disease_D00,secondary_disease_G56,secondary_disease_R73,secondary_disease_C61,secondary_disease_M06,secondary_disease_G95,secondary_disease_D45,secondary_disease_C50,secondary_disease_E68,secondary_disease_S52,secondary_disease_J67,secondary_disease_S43,secondary_disease_B20,secondary_disease_N20,secondary_disease_D50,secondary_disease_G90,secondary_disease_Z35,secondary_disease_D35,secondary_disease_R55,secondary_disease_R04,secondary_disease_I80,secondary_disease_B37,secondary_disease_I07,secondary_disease_C16,secondary_disease_D01,secondary_disease_M13,secondary_disease_E14,secondary_disease_Z03,secondary_disease_I12,secondary_disease_I11,secondary_disease_D42,secondary_disease_I25,secondary_disease_D13,secondary_disease_N40,secondary_disease_M19,secondary_disease_M42
str,str,str,str,str,str,str,str,str,str,list[str],str,list[str],date,str,str,str,list[str],list[date],list[str],i32,i32,str,str,str,i64,i64,i64,f64,i32,i32,list[str],i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""6922520b-d564-48d6-9523-0de4ea…","""Мужской""","""""","""АНАМНЕЗ ЗАБОЛЕВАНИЯ ИБС РАНЕЕ …",""" ЖАЛОБЫ НА МОМЕНТ ОСМОТРА НЕ П…","""{'Название': {0: 'Приемное отд…",""" , Сведения о пребывании пацие…","""ОСН I по Killip.""","""ИБС. Острый инфаркт миокарда н…","""Ожирение 1 степени (ИМТ 31 кг/…","[""I50.1""]","""I21""","[""E66.9""]",2020-12-29,"""Первичный""","""Доставлен бригадой скорой помо…","""Гипертоническая болезнь III ст…","[""I11.9""]",[2023-07-09],"[""I21""]",1,0,"""Улучшение""","""Острое""","""Удовлетворительное""",54,102,null,3.7,1,0,"[""E66.9"", ""I11.9"", ""I50.1""]",3,0,0,0,1,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
"""0A61F963-EC85-4C28-AD6B-63C9D3…","""Мужской""","""""","""АНАМНЕЗ ЗАБОЛЕВАНИЯ КЛИНИКА ИБ…",""" ЖАЛОБЫ НА ОБЩУЮ СЛАБОСТЬ. ""","""{'Название': {0: 'Приемное отд…",""" , Сведения о пребывании пацие…","""ОСН по Killip I""","""ОКС без подъёма сегмента ST. …","""Истинная полицитемия 2А стадия…","[""I50.1""]","""I20""","[""I35.1"", ""I70.2"", … ""I70.8""]",2022-01-16,"""Первичный""","""Доставлен бригадой скорой помо…","""Гипертоническая болезнь III с…","[""I11.9""]",[2022-10-01],"[""I20""]",0,0,"""Улучшение""","""Острое""","""Удовлетворительное""",72,71,null,4.6,0,1,"[""I35.1"", ""I70.2"", … ""I50.1""]",8,1,0,0,1,…,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
"""B646313F-F2FB-4468-8EAA-3F2C38…","""Мужской""","""""","""АНАМНЕЗ ЗАБОЛЕВАНИЯ В ТЕЧЕНИЕ …",""" ЖАЛОБЫ НА МОМЕНТ ОСМОТРА НЕ П…","""{'Название': {0: 'Приемное отд…",""" Электрокардиография (14.02.20…","""Полная БЛНПГ, впервые выявленн…","""Нестабильная стенокардияИБС. Н…",null,"[""I44.7"", ""I50.1""]","""I20""",[],2019-02-14,"""Первичный""","""Плановая госпитализация""","""Сахарный диабет 2 типа. Целево…","[""E11.8"", ""I11.9""]",[2019-03-30],"[""I20""]",0,0,"""Улучшение""","""Острое""","""Удовлетворительное""",59,67,28,3.4,0,1,"[""E11.8"", ""I11.9"", … ""I50.1""]",4,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [16]:
epic_records_with_target["main_desease_status"].value_counts()

main_desease_status,count
str,u32
"""Ранее установленное хроническо…",24
"""Острое""",951
"""None""",16


In [17]:
epic_records_with_target.with_columns(
    pl.concat_str("Состояние " + c("condition_status")).alias("condition_status")
)["condition_status"]

condition_status
str
"""Состояние Удовлетворительное"""
"""Состояние Тяжелое"""
"""Состояние Удовлетворительное"""
"""Состояние Тяжелое"""
"""Состояние Удовлетворительное"""
…
"""Состояние Удовлетворительное"""
"""Состояние Удовлетворительное"""
"""Состояние Удовлетворительное"""


In [18]:
train_data = epic_records_with_target.select(
    pl.concat_str("Пол " + c("gender")).alias("gender"),
    pl.concat_str("Возраст " + c("age").cast(pl.String)).alias("age"),
    pl.concat_str("Состояние " + c("condition_status")).alias("condition_status"),
    "amnez_life",
    "amnez_disease",
    "condition",
    "full_wards",
    pl.concat_str("Основное заболевание " + c("main_disease_desc")).alias(
        "main_disease_desc"
    ),
    "target",
    # остальные не нужны или категориалоьные
).with_columns(
    pl.concat_list(
        pl.selectors.by_dtype(pl.String),
        # separator="\n\n\n",
    ).alias("all_text")
)
train_data.sample(3)

gender,age,condition_status,amnez_life,amnez_disease,condition,full_wards,main_disease_desc,target,all_text
str,str,str,str,str,str,str,str,i32,list[str]
"""Пол Мужской""","""Возраст 50""","""Состояние Удовлетворительное""","""""","""АНАМНЕЗ ЗАБОЛЕВАНИЯ ИБС С 2015…",""" ЖАЛОБЫ НА МОМЕНТ ОСМОТРА НЕ П…",""" , Сведения о пребывании пацие…","""Основное заболевание ИБС. Нест…",1,"[""Пол Мужской"", ""Возраст 50"", … ""Основное заболевание ИБС. Нестабильная стенокардия. Перенесенный инфаркт миокарда нижней стенки ЛЖ (15.11.2015г). Прямое стентирование пр\3 ПМЖВ Resolute Integrity 3,5*30 мм. от 18.10.18г.""]"
"""Пол Женский""","""Возраст 71""","""Состояние Удовлетворительное""","""""","""АНАМНЕЗ ЗАБОЛЕВАНИЯ ИБС РАНЕЕ …",""" ЖАЛОБЫ НА МОМЕНТ ОСМОТРА НЕ П…",""" Рентгеноскопия грудной клетки…","""Основное заболевание ИБС. Нест…",0,"[""Пол Женский"", ""Возраст 71"", … ""Основное заболевание ИБС. Нестабильная стенокардия.""]"
"""Пол Мужской""","""Возраст 60""","""Состояние Удовлетворительное""","""""","""АНАМНЕЗ ЗАБОЛЕВАНИЯ БЕЗ КОРОНА…",""" ЖАЛОБЫ НЕ ПРЕДЪЯВЛЯЕТ. АНГИНО…",""" , Сведения о пребывании пацие…","""Основное заболевание ИБС. Нест…",0,"[""Пол Мужской"", ""Возраст 60"", … ""Основное заболевание ИБС. Нестабильная стенокардия.""]"


In [19]:
for item in train_data["all_text"].to_numpy()[2]:
    pprint(item)
    print()

'Пол Мужской'

'Возраст 62'

'Состояние Удовлетворительное'

''

('АНАМНЕЗ ЗАБОЛЕВАНИЯ КЛИНИКА СТЕНОКАРДИИ С 11 09 2018Г, КОГДА ВПЕРВЫЕ '
 'ПОСТУПИЛ В КО №1 ОКД С Д-ЗОМ ИБС ВПЕРВЫЕ ВОЗНИКШАЯ СТЕНОКАРДИЯ 24 09 2018 '
 'БЫЛО ВЫПОЛНЕНО ПРЯМОЕ СТЕНТИРОВАНИЕ ПР/3 ОВ ST BRIG 2,75 Х 22 ММ 21 03 2019 '
 'ПРОВЕДЕНА КОРОНАРНАЯ БАЛЛОННАЯ АНГИОПЛАСТИКА РЕСТЕНОЗА В СТЕНТЕ ОВ ПАЦИЕНТ '
 'БЫЛ ОБСУЖДЕН НА КЛИНИЧЕСКОМ РАЗБОРЕ, УЧИТЫВАЯ, РЕЗУЛЬТАТ КАГ - РЕСТЕНОЗ '
 'СТЕНТА ОВ, А ТАКЖЕ СТЕНОЗ ВТК 60-70%, ПРИНЯТО РЕШЕНИЕ О ПРОВЕДЕНИИ '
 'КОНТРОЛЬНОЙ КАГ И РЕШЕНИЕ ВОПРОСА О РЕВАСКУЛЯРИЗАЦИИ МИОКАРДА ЧЕРЕЗ 3 МЕСЯЦА '
 'В ПЛАНОВОМ ПОРЯДКЕ ВОЗВРАТ КЛИНИКИ СТЕНОКАРДИИ С МАЯ 2019Г 17 07 2019 В '
 'УСЛОВИЯХ КХО №1 ПРОВЕДЕНА КОНТРОЛЬНАЯ КАГ(СТВОЛ ЛКА: СТЕНОЗИРОВАН В ТЕЛЕ ДО '
 '30 % ДВ: В ПР/3 АРТЕРИЯ СТЕНОЗИРОВАНА НА 60 % ОВ: В ПР/3 ВИЗУАЛИЗИРУЕТСЯ '
 'СТЕНТ, С ПРИЗНАКАМИ РЕСТЕНОЗА ДО 75 - 80 % ВТК: УСТЬЕ - ПР/3 АРТЕРИЯ '
 'СТЕНОЗИРОВАНА НА 70 - 75 % ПКА: В ПР/3 АРТЕРИЯ СТЕНОЗИРОВАНА НА 35 %) 23 07 '
 '19Г ПРОВЕ

### Get embeddings

In [21]:
train_data = train_data.with_columns(
    c("all_text").list.join(" ").alias("all_text_concated")
)

In [75]:
train_data.shape

(991, 11)

In [ ]:
train_data = train_data.with_columns(
    c("all_text").list.join(" ").alias("all_text_concated")
)

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_tokenizer = SentenceTransformer("cointegrated/rubert-tiny2")

In [ ]:
train_data.shape

(991, 11)

In [ ]:
results = []
for row in tqdm(train_data.iter_rows(named=True)):

    encodings = sentence_tokenizer.encode(
        row["all_text_concated"],
        padding=True,
        truncation=True,
        return_tensors="np",
    )
    results.append(encodings)

results = np.array(results)

991it [03:44,  4.41it/s]


In [ ]:
df_for_huggingface = pl.from_numpy(results).with_columns(
    pl.Series(name="target", values=train_data["target"]),
)

### train CatBoost on embeddings

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test, _, _ = train_test_split(
    df_for_huggingface,
    train_data["target"],
    test_size=0.2,
    stratify=train_data["target"],
    random_state=42,
)

In [65]:
train_df, eval_df, _, _ = train_test_split(
    train,
    train["target"],
    test_size=0.2,
    stratify=train["target"],
    random_state=42,
)

In [72]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

In [73]:
model = CatBoostClassifier(
    random_seed=2024,
    # cat_features=cat_cols,
    eval_metric="AUC",
    depth=3,
    iterations=500,
    learning_rate=0.06,
)

model.fit(
    X=train_df.drop("target").to_pandas(),
    y=train_df["target"].to_pandas(),
    early_stopping_rounds=50,
    verbose=True,
    eval_set=(
        eval_df.drop("target").to_pandas(),
        eval_df["target"].to_pandas(),
    ),
)

metric = roc_auc_score(
    test["target"].to_pandas(),
    model.predict_proba(test.drop("target").to_pandas())[:, 1],
)
print(metric.round(3))

0:	test: 0.6098605	best: 0.6098605 (0)	total: 4.98ms	remaining: 2.48s
1:	test: 0.6311826	best: 0.6311826 (1)	total: 8.48ms	remaining: 2.11s
2:	test: 0.6124762	best: 0.6311826 (1)	total: 11.4ms	remaining: 1.89s
3:	test: 0.6247622	best: 0.6311826 (1)	total: 14.6ms	remaining: 1.81s
4:	test: 0.6236525	best: 0.6311826 (1)	total: 17.4ms	remaining: 1.72s
5:	test: 0.6307070	best: 0.6311826 (1)	total: 20.3ms	remaining: 1.67s
6:	test: 0.6321338	best: 0.6321338 (6)	total: 23.5ms	remaining: 1.65s
7:	test: 0.6222257	best: 0.6321338 (6)	total: 26.7ms	remaining: 1.64s
8:	test: 0.6237318	best: 0.6321338 (6)	total: 29.7ms	remaining: 1.62s
9:	test: 0.6217502	best: 0.6321338 (6)	total: 32.6ms	remaining: 1.59s
10:	test: 0.6242866	best: 0.6321338 (6)	total: 35.6ms	remaining: 1.58s
11:	test: 0.6289632	best: 0.6321338 (6)	total: 38.5ms	remaining: 1.56s
12:	test: 0.6260304	best: 0.6321338 (6)	total: 41.4ms	remaining: 1.55s
13:	test: 0.6255549	best: 0.6321338 (6)	total: 44.4ms	remaining: 1.54s
14:	test: 0.6250